# RAG - FSI FAQ with FAISS VectorStore

## Install Install packages and Setup env

In [1]:
install_needed = True  # should only be True once

In [2]:
import sys
import IPython

if install_needed:
    print("installing deps and restarting kernel")
    !{sys.executable} -m pip install -U pip
    !{sys.executable} -m pip install -U sagemaker
    !{sys.executable} -m pip install -U langchain
    !{sys.executable} -m pip install -U faiss-cpu
    
    IPython.Application.instance().kernel.do_shutdown(True)

installing deps and restarting kernel
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


## 1. SageMaker Endpoint Wrapper

In [1]:
import sys
%load_ext autoreload
%autoreload 2
sys.path.append('../utils') # src 폴더 경로 설정

import json
import boto3
import numpy as np
from inference_utils import Prompter
from typing import Any, Dict, List, Optional
from langchain.embeddings import SagemakerEndpointEmbeddings
from langchain.llms.sagemaker_endpoint import LLMContentHandler, SagemakerEndpoint
from langchain.embeddings.sagemaker_endpoint import EmbeddingsContentHandler

### 1.1 [한국어 Text LLM] SageMaker Kullum endpiont handler
- 구름

In [46]:
prompter = Prompter("kullm")
params = {
      'do_sample': False,
      'max_new_tokens': 512, #128
      'temperature': 1.0,  # 0.5 ~ 1.0 default = 1.0 높으면 랜덤하게 자유도. 다음 생성 문장 토큰의 자유도 
      'top_k': 0,
      'top_p': 0.9,
      'return_full_text': False,
      'repetition_penalty': 1.1,
      'presence_penalty': None,
      'eos_token_id': 2
}

class KullmContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs={}) -> bytes:
        '''
        입력 데이터 전처리 후에 리턴
        '''
        context, question = prompt.split("||SPEPERATOR||") 
        prompt = prompter.generate_prompt(question, context)

        #print ("prompt", prompt)
        payload = {
            'prompt': [prompt],
            'params': model_kwargs
        }
                           
        input_str = json.dumps(payload)
        
        return input_str.encode('utf-8')
    

    def transform_output(self, output: bytes) -> str:
        
        response_json = json.loads(output.read().decode("utf-8"))              
        generated_text = response_json[0][0]["generated_text"]
        
        return generated_text    

#### 1.1.a 기존에 deploying한 한국어생성 LLM endpoint 설정
Kullm-polyglot-12-8b-v2-2023-07-25-09-00-53

In [47]:
aws_region = boto3.Session().region_name
LLMTextContentHandler = KullmContentHandler()
endpoint_name_text = "Kullm-polyglot-12-8b-v2-2023-07-25-09-00-53"
seperator = "||SPEPERATOR||"

In [48]:
llm_text = SagemakerEndpoint(
    endpoint_name=endpoint_name_text,
    region_name=aws_region,
    model_kwargs=params,    
    content_handler=LLMTextContentHandler,
)

### 1.2 [한국어 임베딩벡터 모델] SageMaker KoSimCSE-roberta endpiont handler
- KoSimCSE-roberta

In [49]:
class SagemakerEndpointEmbeddingsJumpStart(SagemakerEndpointEmbeddings):
    def embed_documents(self, texts: List[str], chunk_size: int=1) -> List[List[float]]:
        """Compute doc embeddings using a SageMaker Inference Endpoint.

        Args:
            texts: The list of texts to embed.
            chunk_size: The chunk size defines how many input texts will
                be grouped together as request. If None, will use the
                chunk size specified by the class.

        Returns:
            List of embeddings, one for each text.
        """
        results = []
        _chunk_size = len(texts) if chunk_size > len(texts) else chunk_size
        
        print("text size: ", len(texts))
        print("_chunk_size: ", _chunk_size)

        for i in range(0, len(texts), _chunk_size):
            
            #print (i, texts[i : i + _chunk_size])
            response = self._embedding_func(texts[i : i + _chunk_size])
            #print (i, response, len(response[0].shape))
            
            results.extend(response)
        return results

In [50]:
class KoSimCSERobertaContentHandler(EmbeddingsContentHandler):
    
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs={}) -> bytes:
        
        input_str = json.dumps({"inputs": prompt, **model_kwargs})
        
        return input_str.encode("utf-8")

    def transform_output(self, output: bytes) -> str:
        
        response_json = json.loads(output.read().decode("utf-8"))
        ndim = np.array(response_json).ndim    
        
        if ndim == 4:
            # Original shape (1, 1, n, 768)
            emb = response_json[0][0][0]
            emb = np.expand_dims(emb, axis=0).tolist()
        elif ndim == 2:
            # Original shape (n, 1)
            emb = []
            for ele in response_json:
                e = ele[0][0]
                emb.append(e)
        else:
            print(f"Other # of dimension: {ndim}")
            emb = None
        return emb

#### 1.2.a 기존에 deploying한 임베딩 벡터 endpoint 설정
Kullm-polyglot-12-8b-v2-2023-07-25-09-00-53
- 임베딩 핸들러 변수명 : llm_emb 

In [51]:
LLMEmbHandler = KoSimCSERobertaContentHandler()
endpoint_name_emb = "KoSimCSE-roberta-2023-07-26-10-42-42"

In [52]:
llm_emb = SagemakerEndpointEmbeddingsJumpStart(
    endpoint_name=endpoint_name_emb,
    region_name=aws_region,
    content_handler=LLMEmbHandler,
)

**Now, we can build an QA application. <span style="color:red">LangChain makes it extremly simple with following few lines of code</span>.**

# 2.Vector Store - FAISS

- Vector Store : https://python.langchain.com/docs/modules/data_connection/vectorstores/
- FAISS Vector Store : https://python.langchain.com/docs/modules/data_connection/vectorstores/integrations/faiss



In [53]:
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import Chroma, AtlasDB, FAISS
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter

### 2.1 Context file load

#### 2.1.a Increase csv field size to 1억

In [54]:
import csv
csv.field_size_limit(100000000)
csv.field_size_limit()

100000000

#### 2.1.b FAQ CSV 파일 Load

In [55]:
loader = CSVLoader(
    file_path="../dataset/fsi_smart_faq_ko.csv",
    source_column="Source",
    encoding="utf-8"
)
context_documents = loader.load()

#### 2.1.c FSI FAQ Context 데이터 확인 
- 인터넷뱅킹 FAQ > 스마트뱅킹 No.1 ~ N. 89 로 구성되었습니다. 
- https://www.shinhan.com/hpe/index.jsp#050101020000

In [56]:
print(len(context_documents))

context_documents[0:10]

89


[Document(page_content='no: 89\nCategory: 타기관OTP 이용등록방법 알려주세요\nInformation: 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다. \n[경로]\n- 인터넷뱅킹 로그인→ 사용자관리→인터넷뱅킹관리→OTP이용등록  \n- 신한 쏠(SOL) 로그인→ 전체메뉴→설정/인증→ 이용중인 보안매체선택→   OTP이용등록\n \n ※ OTP이용등록후 재로그인을 하셔야 새로 등록된 보안매체가 적용됩니다.\n\n기타 궁금하신 내용은 신한은행 고객센터 1599-8000로 문의하여 주시기 바랍니다.\ntype: 인터넷뱅킹\nSource: 신한은행', metadata={'source': '신한은행', 'row': 0}),
 Document(page_content='no: 88\nCategory: 공동인증서와 금융인증서 차이점이 무엇인가요?\nInformation: 공동인증서 (구 공인인증서)는 용도에 따라 은행/신용카드/보험용 무료 인증서와 전자거래범용(수수료 4,400원) 인증서가 있으며 유효기간은 1년입니다. \n공동인증서는 하드디스크나 이동식디스크, 휴대폰 등 원하시는 기기에 저장해서 이용할 수 있습니다.\n인증서를 저장한 매체에서는 인증서 비밀번호로 편리하게 이용할 수 있으나 다른 기기에서 이용하려면 기기마다 복사하거나 이동식디스크에 저장해서 휴대해야 하는 불편함이 있을 수 있습니다.\n\n금융인증서는 금융인증서는 금융결제원의 클라우드에 저장하여 이용하는 인증서로 발급/이용 시에 클라우드에 접속이 필요합니다.\n금융결제원 클라우드에 연결만 가능하다면 어디서든 편리하게 이용 가능하지만, PC나 USB, 휴대폰 등 다른 기기로 복사는 불가합니다.(유효기간 3년/발급 수수료 무료)\n※ 클라우드 계정 연결을 위해 휴대폰을 통한 ARS, SMS, 마이인포앱 인증 절차가 필요합니다.\ntype: 인증서\nSource: 신한은행', metadata={'source': '신한은행', 'row': 1}),

### 2.2 FAISS 벡터 Indexer 생성

로드한 FAQ 컨텍스트 데이터를 Indexer를 생성 빌드합니다.
- Recursively split by character : https://python.langchain.com/docs/modules/data_connection/document_transformers/text_splitters/recursive_text_splitter

It takes about 1 mins
vectorstore 생성에 1분정도 소요됩니다.

In [57]:
index_creator = VectorstoreIndexCreator(
    vectorstore_cls=FAISS,
    embedding=llm_emb,
    #text_splitter=CharacterTextSplitter(chunk_size=700, chunk_overlap=0),
    #text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=200), 89개짜리가 왜 113개가 되지...
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=200)
)
# 로버타 모델의 청크 크기에 종속되어있음. 청크 파라미터 확인

### 2.2.a 생성 시작

In [59]:
%%time
index = index_creator.from_loaders([loader])

text size:  90
_chunk_size:  1
CPU times: user 4.87 s, sys: 109 ms, total: 4.98 s
Wall time: 17.2 s


### 2.2.b index save and read index from local

In [60]:
# Save
index.vectorstore.save_local("../indexer/fsi_faq_indexer_ko")

In [20]:
# Read
index_ = FAISS.load_local("../indexer/fsi_faq_indexer_ko", llm_emb)

### 2.2.c 생성한 Indexer document 10개 확인

In [61]:
index_.index_to_docstore_id
#index_.docstore._dict[:10]
dict(list(index_.docstore._dict.items())[:10])

{'8930effd-1aed-4d2a-a1d3-f322e61b3140': Document(page_content='no: 89\nCategory: 타기관OTP 이용등록방법 알려주세요\nInformation: 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다. \n[경로]\n- 인터넷뱅킹 로그인→ 사용자관리→인터넷뱅킹관리→OTP이용등록  \n- 신한 쏠(SOL) 로그인→ 전체메뉴→설정/인증→ 이용중인 보안매체선택→   OTP이용등록\n \n ※ OTP이용등록후 재로그인을 하셔야 새로 등록된 보안매체가 적용됩니다.\n\n기타 궁금하신 내용은 신한은행 고객센터 1599-8000로 문의하여 주시기 바랍니다.\ntype: 인터넷뱅킹\nSource: 신한은행', metadata={'source': '신한은행', 'row': 0}),
 'fc75dd31-b98d-4d86-b188-2fc66a066c98': Document(page_content='no: 88\nCategory: 공동인증서와 금융인증서 차이점이 무엇인가요?\nInformation: 공동인증서 (구 공인인증서)는 용도에 따라 은행/신용카드/보험용 무료 인증서와 전자거래범용(수수료 4,400원) 인증서가 있으며 유효기간은 1년입니다. \n공동인증서는 하드디스크나 이동식디스크, 휴대폰 등 원하시는 기기에 저장해서 이용할 수 있습니다.\n인증서를 저장한 매체에서는 인증서 비밀번호로 편리하게 이용할 수 있으나 다른 기기에서 이용하려면 기기마다 복사하거나 이동식디스크에 저장해서 휴대해야 하는 불편함이 있을 수 있습니다.\n\n금융인증서는 금융인증서는 금융결제원의 클라우드에 저장하여 이용하는 인증서로 발급/이용 시에 클라우드에 접속이 필요합니다.\n금융결제원 클라우드에 연결만 가능하다면 어디서든 편리하게 이용 가능하지만, PC나 USB, 휴대폰 등 다른 기기로 복사는 불가합니다.(유효기간 3년/발급 수수료 무료)\n※ 클라우드 계정 연결을 위해 휴대폰을 통한 ARS, SMS, 마이인포앱 

In [62]:
len(list(index_.docstore._dict.items()))

90

## 3. Langchain QnA

In [63]:
from langchain import PromptTemplate
from langchain.chains.question_answering import load_qa_chain


### 3.1 Query and Response

In [64]:
prompt_template = ''.join(["{context}", seperator, "{question}"])
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain = load_qa_chain(llm=llm_text, chain_type="stuff", prompt=PROMPT, verbose=True)



def filter_and_remove_score(res, cutoff_score = 200, variance=1.3):
    # Get the lowest score
    lowest_score = min(score for doc, score in res)
    print('lowest_score : ', lowest_score)
    # If the lowest score is over 200, return an empty list
    if lowest_score > cutoff_score:
        return []
    # Calculate the upper bound for scores
    upper_bound = lowest_score * variance

    
    # Filter the list and remove the score
    res = [doc for doc, score in res if score <= upper_bound]

    return res


def get_similiar_docs(query, k=1, fetch_k=50, score=True, bank=""):

    #print (query)
    
    if score:         
        similar_docs = index_.similarity_search_with_score(
            query,
            k=k,
            fetch_k=fetch_k,
            filter=dict(source=bank)
        )
        similar_docs=filter_and_remove_score(similar_docs)
    else:        
        similar_docs = index_.similarity_search(
            query,
            k=k,
            fetch_k=fetch_k,
            filter=dict(source=bank)
        )
        
    return similar_docs

def get_answer(query, bank="", k=1):
                
    search_query = query
    
    #criteria = '만약,'+query+' 카테고리를 못찾으면 설명 하지마세요.'
    #llm_query_with_criteria = llm_query+criteria
    similar_docs = get_similiar_docs(search_query, k=k, bank=bank)
    
    print("similar_docs : ", similar_docs)
    llm_query = '고객 서비스 센터 직원처럼, '+query+' 카테고리에 대한 Information을 찾아서 설명해주세요.'
    
    if not similar_docs:
        llm_query = query

    answer = chain.run(input_documents=similar_docs, question=llm_query)
    
    return answer



def get_admin_answer (query, bank="", k=1):
    search_query = query
    similar_docs = get_similiar_docs(search_query, k=k, bank=bank)
    llm_query = 'Document 입력 문장들을 보고서 형식으로 바꿔줘.'

    llm_answer = chain.run(input_documents=similar_docs, question=llm_query)
    return llm_answer

In [65]:
%%time
question = "안녕하세요. 날씨가 참 좋네요."
response = get_answer(question, bank="신한은행", k=5)

print (f'question: {question}')
print('==========================')
print('\n')
print (f'response: \n {response}')

lowest_score :  293.9933
similar_docs :  []


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
||SPEPERATOR||안녕하세요. 날씨가 참 좋네요.

> Finished chain.

> Finished chain.
question: 안녕하세요. 날씨가 참 좋네요.


response: 
 안녕하세요! 오늘은 무엇을 도와드릴까요?
CPU times: user 48.5 ms, sys: 230 µs, total: 48.7 ms
Wall time: 766 ms


### 3.1.a FAQ 데이터 응답문장 테스트

In [68]:
%%time
question = "타기관OTP 등록 방법 알려주세요"
response = get_answer(question, bank="신한은행", k=3)

print (f'question: {question}')
print('==========================')
print('\n')
print (f'response: \n {response}')

lowest_score :  112.327644
similar_docs :  [Document(page_content='no: 89\nCategory: 타기관OTP 이용등록방법 알려주세요\nInformation: 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다. \n[경로]\n- 인터넷뱅킹 로그인→ 사용자관리→인터넷뱅킹관리→OTP이용등록  \n- 신한 쏠(SOL) 로그인→ 전체메뉴→설정/인증→ 이용중인 보안매체선택→   OTP이용등록\n \n ※ OTP이용등록후 재로그인을 하셔야 새로 등록된 보안매체가 적용됩니다.\n\n기타 궁금하신 내용은 신한은행 고객센터 1599-8000로 문의하여 주시기 바랍니다.\ntype: 인터넷뱅킹\nSource: 신한은행', metadata={'source': '신한은행', 'row': 0})]


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
no: 89
Category: 타기관OTP 이용등록방법 알려주세요
Information: 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다. 
[경로]
- 인터넷뱅킹 로그인→ 사용자관리→인터넷뱅킹관리→OTP이용등록  
- 신한 쏠(SOL) 로그인→ 전체메뉴→설정/인증→ 이용중인 보안매체선택→   OTP이용등록
 
 ※ OTP이용등록후 재로그인을 하셔야 새로 등록된 보안매체가 적용됩니다.

기타 궁금하신 내용은 신한은행 고객센터 1599-8000로 문의하여 주시기 바랍니다.
type: 인터넷뱅킹
Source: 신한은행||SPEPERATOR||고객 서비스 센터 직원처럼, 타기관OTP 등록 방법 알려주세요 카테고리에 대한 Information을 찾아서 설명해주세요.

> Finished chain.

> Finished chain.
question: 타기관OTP 등록 방법 알려주세요


res

In [67]:
%%time
question = "뱅크아이디란 무엇인가요?"
response = get_answer(question, bank="신한은행", k=3)

print (f'question: {question}')
print('==========================')
print('\n')
print (f'response: \n {response}')



lowest_score :  153.17928
similar_docs :  [Document(page_content='no: 83\nCategory: 뱅크아이디란 무엇인가요?\nInformation: 뱅크아이디란, 블록체인기반의 은행권 공동인증서비스로 블록체인 원장을 분산하여 저장/관리함으로써 정보의 신뢰성과 안정성을 높이는 인증기술방식입니다. 기존의 뱅크사인(BankSign)이  뱅크아이디(마이인포) 로 전환되었으며 별도의 앱 설치 없이 신한 쏠(SOL)에서 직접 신청 및 관리가 가능합니다.\ntype: 뱅크아이디\nSource: 신한은행', metadata={'source': '신한은행', 'row': 6}), Document(page_content='no: 31\nCategory: 보안카드 번호 입력은 어떻게 하면 되죠?\nInformation: 씨크리트(보안)카드는 고객님이 은행에서 신한온라인서비스(인터넷뱅킹)에 가입하신 후 받은 카드입니다.  보안카드번호가 필요한 모든 거래에  기타 궁금하신 내용은 신한은행 고객센터 1599-8000로 문의하여 주시기 바랍니다.\ntype: 인터넷뱅킹\nSource: 신한은행', metadata={'source': '신한은행', 'row': 58}), Document(page_content="no: 35\nCategory: 로그인은 어디서 하나요?\nInformation: 인터넷뱅킹에 로그인하시려면 신한은행 홈페이지에서 개인 → 뱅킹로그인 버튼을 클릭하시면 됩니다. 참고로 인터넷뱅킹에 로그인하시려면 우선 공동인증서 또는 금융인증서를 발급받으셔야 하며, '인증센터'에서 발급받으실 수 있습니다. 기타 궁금하신 내용은 신한은행 고객센터 1599-8000로 문의하여 주시기 바랍니다.\ntype: 인터넷뱅킹\nSource: 신한은행", metadata={'source': '신한은행', 'row': 54})]


> Entering new StuffDocumentsChain chain...


> Entering new LLMC

In [33]:

%%time
question = "화면에서 미래를함께하는따뜻한금융 안전한금융거래를위해준비중입니다. 라고 나오고 맨날 멈추는데 어떻게 하냐? 아니쓸수가 없자나"
response = get_answer(question, bank="신한은행", k=1)

print (f'question: {question}')
print('==========================')
print('\n')
print (f'response: \n {response}')



lowest_score :  180.90964


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
no: 78
Category: 미래를함께하는따뜻한금융 안전한금융거래를위해준비중입니다.' 화면에서 멈춤 / 로딩중 멈추는데 어떻게 하나요?
Information: 오류 사유는 인터넷 익스플로러 설정값의 영향이거나 이용하시는 장소의 네트워크 영향에 의해 로딩이 원활하지 않은 경우입니다. 
조치방법은 
1. 임시 인터넷 파일 및 쿠키 삭제, 신뢰할 수 있는 사이트 추가 
 인터넷 익스플로러 상단의 [도구] > [인터넷옵션]에서
① [일반Tab] > 검색기록항목의 [삭제] 버튼 클릭 "임시 인터넷 파일 및 사이트 파일" , "쿠키 및 웹사이트 데이터" 항목만 체크하고 [삭제] 
② [보안Tab] > 신뢰할 수 있는 사이트 > [사이트] 클릭하여 [ https://*.shinhan.com ] 추가 후 적용 및 확인하십시오. 
 또는, 당행 보안프로그램 삭제 열려있는 브라우저 종료 후, 시작 > 제어판의 [프로그램 추가/제거] (총 4개이며 보이지 않으면 제거할 필요없음) 
- AhnLab Safe Transaction, 
 - iniLINE CrossEX Service, 
- INISAFE CrossWeb EX v3.0, 
- TouchEn nxKey with E2E for 32bit
3. 새 브라우저 열어서 개인인터넷뱅킹 재접속 후 [통합설치] 진행 하세요. 
기타 궁금하신 내용은 신한은행 고객센터 1599-8000로 문의하여 주시기 바랍니다.
type: 
Source: 신한은행||SPEPERATOR||고객 서비스 센터 직원처럼, 화면에서 미래를함께하는따뜻한금융 안전한금융거래를위해준비중입니다. 라고 나오고 맨날 멈추는데 어떻게 하냐? 아니쓸수가 없자나 카테고리에 대한 Information을 찾아서 설명해주세요.

> Finished c

In [37]:

%%time
question = "증권사 인증서는 사용 못하나요?"
response = get_answer(question, bank="신한은행", k=3)

print (f'question: {question}')
print('==========================')
print('\n')
print (f'response: \n {response}')



lowest_score :  121.71279


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
no: 79
Category: 증권사에서 발급받은 인증서는 은행에서 사용 불가한가요?
Information: 인증서 용도로 구분해 주셔야 합니다. 
증권용(무료) 인증서를 통한 은행 거래는 불가하나, 발급하신 인증서가 전자거래범용(유료) 인증서일 경우 은행에서도 이용 가능합니다.
type: 인증서
Source: 신한은행||SPEPERATOR||고객 서비스 센터 직원처럼, 증권사 인증서는 사용 못하나요? 카테고리에 대한 Information을 찾아서 설명해주세요.

> Finished chain.

> Finished chain.
question: 증권사 인증서는 사용 못하나요?


response: 
 증권사 인증서는 은행에서 사용할 수 없습니다. 그러나 무료 증권용 인증서를 발급 받으신 경우, 은행 거래가 불가능하지만, 전자거래범용 인증서를 발급 받으신 경우 은행에서도 이용 가능합니다.
CPU times: user 9.11 ms, sys: 596 µs, total: 9.7 ms
Wall time: 2.22 s


In [38]:

%%time
question = "공동인증서랑 금융인증서 같이 쓸수있나요?"
response = get_answer(question, bank="신한은행", k=3)

print (f'question: {question}')
print('==========================')
print('\n')
print (f'response: \n {response}')



lowest_score :  85.2139


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
no: 84
Category: 기존 공동인증서를 보유한 상태에서 금융인증서 발급이 가능한가요?
Information: 공동인증서와 금융인증서는 별개의 인증서로 두 가지 인증서를 모두 사용할 수 있습니다.
type: 금융인증서
Source: 신한은행||SPEPERATOR||고객 서비스 센터 직원처럼, 공동인증서랑 금융인증서 같이 쓸수있나요? 카테고리에 대한 Information을 찾아서 설명해주세요.

> Finished chain.

> Finished chain.
question: 공동인증서랑 금융인증서 같이 쓸수있나요?


response: 
 예, 공동인증서와 금융인증서는 별도의 인증서입니다. 공동인증서는 은행 및 금융 기관과 같은 다양한 웹사이트에서 사용자 신원을 확인하고 로그인할 때 사용됩니다. 반면에 금융인증서는 전자상거래, 온라인 뱅킹 등 금융 거래 시 개인정보 보호 및 보안을 강화하기 위해 사용되며, 금융기관에서 발급합니다.
CPU times: user 12.5 ms, sys: 0 ns, total: 12.5 ms
Wall time: 3.64 s


In [34]:

%%time
question = "홈페이지 탈퇴하는방법 알려줘"
response = get_answer(question, bank="신한은행", k=3)

print (f'question: {question}')
print('==========================')
print('\n')
print (f'response: \n {response}')





> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
no: 48
Category: 홈페이지 탈회하고 싶습니다. 어떻게 하나요?
Information: 홈페이지 로그인 후 회원탈퇴를 하실 수 있습니다. 홈페이지 상단에 있는 고객센터 > 회원서비스 > 회원탈퇴 메뉴에서 회원탈퇴 진행할 수 있습니다. 혹시, 비밀번호를 분실하셨다면 ID/PW 로그인 화면 하단 비밀번호 재설정 메뉴를 통해서 비밀번호 재설정하고 로그인 후 회원탈퇴 가능합니다. 기타 문의는 신한은행고객센터 (☎ 1599-8000번)으로 문의 바랍니다.
type: 홈페이지
Source: 신한은행

no: 17
Category: 인터넷뱅킹 해지 방법 알려주세요
Information: [경로] - 인터넷뱅킹 로그인 → 사용자관리 → 인터넷뱅킹관리 → 인터넷뱅킹해지 ☞ 신한은행에서 발급한 인증서 폐기여부 선택가능합니다. ☞ 쏠(모바일뱅킹) 도 사용할수 없음 - 신한 쏠(SOL) 로그인 → 전체메뉴 → 설정/인증 → 서비스 해지 ☞ 쏠(SOL)만 해지 또는 인터넷뱅킹&쏠(SOL) 모두 해지 중 선택하여 해지가능합니다. 기타 궁금하신 내용은 신한은행 고객센터 1599-8000로 문의하여 주시기 바랍니다.
type: 인터넷뱅킹
Source: 신한은행

no: 24
Category: 인터넷뱅킹 자동이체해지 방법안내
Information: 인터넷뱅킹에 로그인하신후, 좌측의 메뉴항목에서 이체 > 자동이체 > 자동이체조회 변경/취소 서비스를 통해 해지하실 수 있습니다.
type: 인터넷뱅킹
Source: 신한은행||SPEPERATOR||고객 서비스 센터 직원처럼, 홈페이지 탈퇴하는방법 알려줘 카테고리에 대한 Information을 찾아서 설명해주세요.

> Finished chain.

> Finished chain.
question: 홈페이지 탈퇴하는방법 알려줘


response: 


In [45]:

%%time
question = "질문 답변 감사합니다. 좋은 하루 되세요."
response = get_answer(question, bank="신한은행", k=3)

print (f'question: {question}')
print('==========================')
print('\n')
print (f'response: \n {response}')



lowest_score :  236.9253
similar_docs :  []


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
||SPEPERATOR||질문 답변 감사합니다. 좋은 하루 되세요.

> Finished chain.

> Finished chain.
question: 질문 답변 감사합니다. 좋은 하루 되세요.


response: 
 천만에요! 질문에 도움을 드릴 수 있어서 기쁩니다. 다른 질문이 있으시면 언제든지 문의해 주세요. 제가 도와드리겠습니다!
CPU times: user 7.99 ms, sys: 1.16 ms, total: 9.15 ms
Wall time: 1.69 s
